In [1]:
from PIL import Image
import glob
import os
import shutil

# Image Resizing

In [2]:
def resize_image(img, path, i):
    # Resize and save image
    image = Image.open(img)
    image = image.resize((150,150))
    image.save(path+'/'+str(i)+'.jpg')
    return
        
def create_directory(path):
    # Create directory for resized images
    if os.path.isdir(path) == True:
        print ("directory exists: " + path)
        print ('deleting directory: ' + path)
        shutil.rmtree(path)
        os.mkdir(path)
        print ("directory created: " + path)
    else :
        os.mkdir(path)
        print ("directory created: " + path)
    return

def resize_images():
    directories = glob.glob('flower_photos/*/')
    path = os.path.join('resized')
    # shutil.rmtree(path)
    create_directory(path)

    for directory in directories:
        flower_type = str.split(directory,'/')[1]    
       # path = os.path.join(path1,'resized',flower_type)
        path2 = os.path.join('resized',flower_type)
        create_directory(path2)
        i = 0
        images= glob.glob(directory+'/*.jpg')
        for image in images:
            resize_image(image, path2 , i)
            i += 1
    shutil.rmtree('flower_photos')
    return

%%time
!curl -O http://download.tensorflow.org/example_images/flower_photos.tgz
!tar -xf flower_photos.tgz
os.remove("flower_photos.tgz") 
resize_images()

# Split Images Train-Test

In [3]:
directories = glob.glob('resized/*/')

In [4]:
import pathlib
import tensorflow as tf

2021-10-19 22:13:15.394542: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-19 22:13:15.394603: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
data_dir = 'resized'
# data_dir = pathlib.Path(data_dir)
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    validation_split=0.25,
    subset="training",
    seed=123,
    image_size=(150, 150),
    batch_size=32)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    validation_split=0.25,
    subset="validation",
    seed=123,
    image_size=(150, 150),
    batch_size=32)

Found 3670 files belonging to 5 classes.
Using 2753 files for training.


2021-10-19 22:13:19.063963: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-10-19 22:13:19.064028: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-19 22:13:19.064080: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sarthak-MacBookPro): /proc/driver/nvidia/version does not exist


Found 3670 files belonging to 5 classes.
Using 917 files for validation.


In [6]:
class_names = train_ds.class_names
print(class_names)

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


# Model VGG16

In [7]:
image_w, image_h = 150, 150

import warnings
warnings.simplefilter("ignore")

In [8]:
# model = tf.keras.applications.VGG16(
#     include_top=False,
#     weights="imagenet",
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=5,
#     classifier_activation="softmax",
# )

model = tf.keras.applications.VGG16(
    include_top=True,
    weights="imagenet",
    classifier_activation="softmax",
)


# VGG16 pre-trained model without fully connected layers and with different input dimensions
model = tf.keras.applications.VGG16(
    weights = "imagenet", 
    include_top=False, 
    input_shape = (image_w, image_h, 3)
)

model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [9]:
# Freezing the layers we don't want to train (here I freeze  until the end of block 2 = 7 layers)
for layer in model.layers:
    layer.trainable = False
    
for i, layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable)
    
# model.summary()

0 input_2 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 False
16 block5_conv2 False
17 block5_conv3 False
18 block5_pool False


In [10]:
# Adding custom layers to create a new model 
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D

new_model = Sequential([
    model,
    Flatten(name='flatten'),
    Dense(256, activation='relu', name='new_fc1', kernel_initializer="HeNormal"),
    Dense(5, activation='softmax', name='new_predictions')
])

new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
new_fc1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
new_predictions (Dense)      (None, 5)                 1285      
Total params: 16,813,381
Trainable params: 2,098,693
Non-trainable params: 14,714,688
_________________________________________________________________


In [11]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001,)

new_model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    optimizer = optimizer, 
    metrics=["accuracy"])

In [ ]:
%%time
new_model.fit(train_ds, validation_data=val_ds, epochs=10)

Epoch 1/10


2021-10-19 22:13:31.663496: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


50/87 [================>.............] - ETA: 13:35 - loss: 5.1678 - accuracy: 0.6513